In [240]:
import numpy as np
from fvcore.nn import giou_loss

height = 1024
width = 2048

n_box1, n_box2 = np.random.randint(18)+2, np.random.randint(18)+2



boxes1, boxes2 = [], []
for i in range(n_box1):
    [y0, y1] = sorted([np.random.randint(height), np.random.randint(height)])
    [x0, x1] = sorted([np.random.randint(width), np.random.randint(width)])
    box = [x0, y0, x1, y1]
    boxes1.append(box)
for i in range(n_box2):
    [y0, y1] = sorted([np.random.randint(height), np.random.randint(height)])
    [x0, x1] = sorted([np.random.randint(width), np.random.randint(width)])
    box = [x0, y0, x1, y1]
    boxes2.append(box)
    


In [241]:
import torch
from detectron2.structures import pairwise_iou, Boxes, BoxMode

from scipy.optimize import linear_sum_assignment
from modelSeleTools_DAS.methodsDirectory2Fast import IoUCost
from torchvision.ops import complete_box_iou_loss
from fvcore.nn.giou_loss import giou_loss

iou_loss = IoUCost(iou_mode="iou")

boxes1 =torch.Tensor(boxes1)
boxes2 =torch.Tensor(boxes2)

sampleCnt, _ = boxes1.shape
sampleCntPerturb, _ = boxes2.shape
max_match = min(sampleCnt, sampleCntPerturb)

#match_quality_matrix = pairwise_iou(Boxes(boxes1), Boxes(boxes2)) # gt rows, perturb cols
#match_row_iou, match_col_iou = linear_sum_assignment(match_quality_matrix, maximize=True)

pairwise_iou_matrix = pairwise_iou(Boxes(boxes1), Boxes(boxes2)) # gt rows, perturb cols
match_row_pairwise_iou, match_col_pairwise_iou = linear_sum_assignment(pairwise_iou_matrix, maximize=True)
pairwise_iou_ = pairwise_iou_matrix[match_row_pairwise_iou, match_col_pairwise_iou].sum().numpy().tolist() / max_match

complete_box_iou = complete_box_iou_loss(torch.Tensor(boxes1[match_row_pairwise_iou,:]), torch.Tensor(boxes2[match_col_pairwise_iou, :]), reduction="mean").numpy()
giou = -giou_loss(torch.Tensor(boxes1[match_row_pairwise_iou,:]), torch.Tensor(boxes2[match_col_pairwise_iou, :]), reduction="mean").item()



iou_cost_matrix = iou_loss(boxes1, boxes2)
matched_rowIdx, matched_colIdx = linear_sum_assignment(iou_cost_matrix)
iou_cost = -iou_cost_matrix[matched_rowIdx, matched_colIdx].sum().numpy().tolist() / max_match
print(f"pairwise iou: {pairwise_iou_}, iou_cost: {iou_cost}, complete_box_iou: {complete_box_iou}, giou: {giou}")

pairwise iou: 0.37308669090270996, iou_cost: 0.37308669090270996, complete_box_iou: 0.7557662725448608, giou: -0.8724836111068726


In [242]:
torch.Tensor(boxes2[match_col_pairwise_iou,:])

tensor([[ 763.,  798.,  876.,  899.],
        [ 391.,  131.,  646.,  703.],
        [ 791.,  466., 2040.,  887.],
        [ 894.,  247., 1402.,  767.],
        [ 366.,  295.,  790.,  610.]])

In [243]:
print(Boxes(boxes1).area())
print(Boxes(boxes2).area())

tensor([ 59290.,  12834., 175051.,  83104.,  68783., 299088.,   4995., 313632.,
          8064.,  75225., 132000.])
tensor([ 11413., 264160., 525829., 145860., 133560.])


## Differences between pairwise_iou and complete_box_iou_loss

In [244]:
diff = matched_colIdx - match_row_pairwise_iou
print(diff)
diff = matched_rowIdx - match_col_pairwise_iou
print(diff)


[-1  1 -3 -6 -6]
[ 1 -1  3  6  6]
